In [1]:
from import_libraries.project_functions import *

warnings.filterwarnings("ignore")

# data from yfinance

In [2]:
def get_historical_data(timeframe=mt5.TIMEFRAME_D1, symbol="GER30", count = 30_000):
    """
    Получает исторические данные из MetaTrader 5.

    Аргументы:
    timeframe (int): Временной интервал для исторических данных.
    symbol (str): Символ, для которого необходимо получить исторические данные.
    start (str): Начальная дата для получения исторических данных.

    Возвращает:
    pd.DataFrame: DataFrame, содержащий полученные исторические данные.
    """
    if not connect_to_mt5():
        return None

    
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, count)

    # Проверяем успешность получения данных
    if rates is not None:
        # Конвертируем данные в pandas DataFrame
        rates_frame = pd.DataFrame(rates)
        rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
        rates_frame = rates_frame.rename(columns=lambda x: x.capitalize())
        rates_frame = rates_frame.rename(columns={"Time": "Date"})
        
        rates_frame['Volume'] = rates_frame['High'] - rates_frame['Low']
        rates_frame['MaxPositivePriceChange'] = rates_frame['High'] - rates_frame['Open'] 
        rates_frame['MaxNegativePriceChange'] = rates_frame['Open'] - rates_frame['Low']
        
        rates_frame['PriceChange'] = abs(rates_frame['Close'] - rates_frame['Close'].shift(1))
        
        # Закрываем соединение с MetaTrader 5
        mt5.shutdown()
        return rates_frame
    else:
        print("Ошибка при получении данных.")
        mt5.shutdown()
        return None



In [3]:
data = get_historical_data(count = 70_00) # D1
# data_m5 = get_historical_data(timeframe=mt5.TIMEFRAME_M5, symbol="GER30", start='2022-01-01')
data.shape

Połączono z kontem


(3659, 12)

In [4]:
data

,Date,Open,High,Low,Close,Tick_volume,Spread,Real_volume,Volume,MaxPositivePriceChange,MaxNegativePriceChange,PriceChange
0,2009-06-29,4748.0,4905.5,4741.0,4884.5,1926,0,0,164.5,157.5,7.0,NaN
1,2009-06-30,4884.5,4908.0,4790.5,4819.5,1701,0,0,117.5,23.5,94.0,65.0
2,2009-07-01,4819.5,4931.5,4807.0,4895.5,1731,0,0,124.5,112.0,12.5,76.0
3,2009-07-02,4862.0,4881.5,4705.0,4731.5,2226,0,0,176.5,19.5,157.0,164.0
4,2009-07-03,4734.5,4751.5,4685.5,4696.0,936,0,0,66.0,17.0,49.0,35.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3654,2023-11-02,15074.9,15293.9,15069.8,15292.9,21900,13,0,224.1,219.0,5.1,234.0
3655,2023-11-03,15276.9,15336.9,15217.9,15265.9,22804,13,0,119.0,60.0,59.0,27.0
3656,2023-11-06,15258.8,15299.9,15166.9,15208.8,17530,13,0,133.0,41.1,91.9,57.1
3657,2023-11-07,15156.9,15260.9,15132.9,15218.9,18870,13,0,128.0,104.0,24.0,10.1


In [5]:
# data.hist(figsize = (10,10));

# data preprocessing

In [6]:
# preprocessor = Preprocessing_stock_data(data)
# all_indicator = preprocessor.all_methods()

# data = preprocessor.support_resistance_line()
# data

In [7]:
# all_indicator.sample(3)

# experiments 



Etap I. Określanie kierunku.
1. Wyznaczam cenę max i min dla różnych przedziałów od 1 do 60 dni wstecz.
2. Określam czy cena w bieżącym dniu przekroczyła wyznaczony poziom min lub max.
3. Określam czy cena w bieżącym dniu powróciła do określonego przedziału między min a max.
4. Wyznaczam bieżący kieru5.ek:
- po przekroczeniu i powrocie do poziomu min ("wsparcie") kierun  to6.BUY;
- po przekroczeniu i powrocie do poziomu max ("opór") kieru ek to7SELL;
5. Po zakończeniu sesji powtarzam punk
8. Sprawdzam w którym kierunku następnego dnia był większy ruch. W górę to cena max - otwarcie. W dół to cena otwarcie  - min.ty 1-4 d

Marcin Araszkiewicz19:34

zmiana po sell = close po sell - close po buy

zmiana po buy = close po sell - close po buy




НАПИСАТЬ НОВЫЙ КАЛЬКУЛЯТОР 

In [8]:
# def calculate_accumulated_price_changes (data, princ = False):    
#     df = data.copy()
    
#     signal_changes = df['Signal'].ne(df['Signal'].shift())
    
#     indices = df.index[signal_changes].tolist()
    
#     df['AccumulatedPriceChange'] = 0.0
#     accumulated_change = 0.0
    
#     for i in range(0, len(indices) - 1):
#         if df.loc[indices[i], 'Signal'] == 'sell' and df.loc[indices[i + 1], 'Signal'] == 'buy':
#             start_sell_index = indices[i]
#             end_buy_index = indices[i + 1]
    
#             start_sell_price = df.loc[start_sell_index, 'Open']
#             end_buy_price = df.loc[end_buy_index, 'Open']
    
#             price_change_sell_to_buy = end_buy_price - start_sell_price
    
#             accumulated_change += price_change_sell_to_buy
#             df.loc[indices[i + 1], 'AccumulatedPriceChange'] = accumulated_change
    
#         elif df.loc[indices[i], 'Signal'] == 'buy' and df.loc[indices[i + 1], 'Signal'] == 'sell':
#             start_buy_index = indices[i]
#             end_sell_index = indices[i + 1]
    
#             start_buy_price = df.loc[start_buy_index, 'Open']
#             end_sell_price = df.loc[end_sell_index, 'Open']
    
#             price_change_buy_to_sell = end_sell_price - start_buy_price
    
#             accumulated_change += price_change_buy_to_sell
#             df.loc[indices[i + 1], 'AccumulatedPriceChange'] = accumulated_change
    
#     df["AccumulatedPriceChange"] = df["AccumulatedPriceChange"].shift(-1).fillna(0)
#     all_sell = df.loc[df["Signal"] == "sell", "AccumulatedPriceChange"].sum()
#     all_buy = df.loc[df["Signal"] == "buy", "AccumulatedPriceChange"].sum()

#     if princ:
#         return all_sell, all_buy
#     else:
#         return all_sell + all_buy



In [ ]:
rebound_best_parameters = optimize_parameters(data, rebound_analysis, calculate_accumulated_price_changes,
                                              windows_size= [i for i in range(1, 366, 1)],
                                              bias= [i for i in range(1, 50, 1)]
                                                )

print(f"Najlepsze opcje: {rebound_best_parameters}")

# [10:08<00:00, 1.59s/it]
# Najlepsze opcje: {'window_size': 1, 'bias': 4}

  0%|          | 0/365 [00:00<?, ?it/s]

In [ ]:
current_data = define_level(data, 1, 4)
rebound_data = rebound_analysis(current_data)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

In [ ]:
# Tworzenie kopii danych, aby nie modyfikować oryginalnych danych
df = rebound_data.copy()

# Obliczanie Breakout_from_Resistance - oznacza przełamanie od oporu w górę
# Przy użyciu warunków, sprawdzamy, czy RollingMax jest mniejszy od High
# i jednocześnie większy od Close, co wskazuje na przełamanie od oporu w górę.
# Następnie konwertujemy wynik na wartość całkowitą (0 lub 1).
df['Breakout_from_Resistance'] = (
    (df["RollingMax"] < df["High"]) & 
    (df["RollingMax"] > df["Close"])
).astype(int)

# Obliczanie Breakout_from_Support - oznacza przełamanie od wsparcia w górę
# Podobnie jak wyżej, sprawdzamy warunki, aby znaleźć przełamanie od wsparcia w górę.
df['Breakout_from_Support'] = (
    (df["RollingMin"] > df["Low"]) & 
    (df["RollingMin"] < df["Close"])
).astype(int)

In [ ]:
breakdown_best_parameters = optimize_parameters(data, breakdown_analysis, calculate_accumulated_price_changes,
                                                windows_size= [i for i in range(1, 366, 1)],
                                                  bias= [i for i in range(1, 50, 1)]
                                                )

print(f"Najlepsze opcje: {breakdown_best_parameters}")


In [ ]:
current_data = define_level(data, 1, 1)
rebound_data = breakdown_analysis(current_data)
current_score = calculate_accumulated_price_changes(rebound_data, princ=True)
current_score

In [ ]:
df["RollingMax"] = rebound_data["RollingMax"]
df["RollingMin"] = rebound_data["RollingMin"]


# Obliczanie Breakdown_from_Resistance - oznacza przełamanie od oporu w dół
# Tutaj sprawdzamy, czy RollingMax jest mniejszy od High i jednocześnie 
# mniejszy od Close, co wskazuje na przełamanie od oporu w dół.
df['Breakdown_from_Resistance'] = (
    (df["RollingMax"] < df["High"]) & 
    (df["RollingMax"] < df["Close"])
).astype(int)

# Obliczanie Breakdown_from_Support - oznacza przełamanie od wsparcia w dół
# Podobnie jak wyżej, sprawdzamy warunki, aby znaleźć przełamanie od wsparcia w dół.
df['Breakdown_from_Support'] = (
    (df["RollingMin"] > df["Low"]) & 
    (df["RollingMin"] > df["Close"])
).astype(int)

# Wyświetlanie sumy wartości
print("Breakout_from_Resistance-> ", df['Breakout_from_Resistance'].sum())
print("Breakout_from_Support-> ", df['Breakout_from_Support'].sum())
print("Breakdown_from_Resistance-> ", df['Breakdown_from_Resistance'].sum())
print("Breakdown_from_Support-> ", df['Breakdown_from_Support'].sum())

In [ ]:
df[["Breakout_from_Resistance","Breakout_from_Support","Breakdown_from_Resistance","Breakdown_from_Support"]].describe()  

df.cumsum()


# AI

In [ ]:
def create_dataset(dataset, time_step=25):
    dataX, dataY = [], []
    dataset_without_close = dataset.drop('Close', axis=1)

    for i in range(len(dataset) - time_step - 1):
        if (i + time_step) < len(dataset):
            a = dataset_without_close.iloc[i:(i + time_step), :]
            dataX.append(a.values)
            dataY.append(dataset.iloc[i + time_step]['Close'])  # Добавлено изменение в индексации здесь
    return np.array(dataX), np.array(dataY)

In [ ]:
def create_dataset(dataset, time_step):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[i + time_step, 0])
    return np.array(dataX), np.array(dataY)